### Import

In [12]:
import os 
import pymysql
import pandas as pd
import datetime as dt
from tqdm import tqdm
from konlpy.tag import Komoran, Kkma, Okt, Hannanum

### 고유명사 추출 함수

In [13]:
komoran = Komoran()
kkma = Kkma()
okt = Okt()
hannanum = Hannanum()

In [32]:
def find_NNP(text, model):
    result = set()
    pos = model.pos(text)
    for i in tqdm(range(len(pos))):
        if pos[i][1] in ("SL", "OL", "Alpha", "Foreign", "F")  and len(pos[i][0]) > 1:
            result.add(pos[i][0])
        elif pos[i][1] in ('NNP', "Noun", "NQ", "N", "NNG"):
            result.add(pos[i][0])
    print()
    return result

In [33]:
text = '네이버, NHK 한국어 동시통역 문자 생중계 갤럭시 S23 삼성전자 이재용'
komoran_result = find_NNP(text, komoran)
print(komoran_result)
kkma_result = find_NNP(text, kkma)
print(kkma_result)
okt_result = find_NNP(text, okt)
print(okt_result)

100%|██████████| 13/13 [00:00<00:00, 13063.24it/s]



{'네이버', '이재용', '중계', '23', '삼성전자', '한국어', '동시통역', '갤럭시', '문자', 'NHK'}


100%|██████████| 19/19 [00:00<?, ?it/s]



{'네이버', '이재', '중계', '시', '통역', '동시', '삼성', '전자', '한국어', '문자', 'NHK'}


100%|██████████| 14/14 [00:00<?, ?it/s]


{'네이버', '이재용', '통역', '동시', '삼성', '생중계', '한국어', '전자', '갤럭시', '문자', 'NHK'}


### Data 들고오기! 함수

In [21]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='modoostock', charset='utf8mb4', autocommit=True)
news_list = pd.read_sql(sql="SELECT * FROM news WHERE company_id=2;", con=con, index_col="id")
con.close()

In [29]:
def mysql_to_dataframe(company_id):
  con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='modoostock', charset='utf8mb4', autocommit=True)
  df_news = pd.read_sql(sql="SELECT * FROM news WHERE company_id={};".format(company_id), con=con, index_col="id")
  return df_news

### 고유 명사 추출

- 모든 기사 제목을 합쳐 형태소 분석 : NNP(고유명사), SL/OL(외국어)
- 이후 기업명, 사람 이름, 제품명을 수동으로 마스킹

In [39]:
def company_name_replace(input):
  return input.replace("NAVER", "네이버") \
              .replace("삼성 전자", "삼성전자") \
              .replace("SAMSUNG", "삼성전자") \
              .replace("SAM SUMG", "삼성전자") \
              .replace("LG 화학", "LG화학") \
              .replace("LG 전자", "LG전자") \
              .replace("GC녹십자웰빙", "녹십자") \
              .replace("GC녹십자", "녹십자") \
              .replace("녹십자생명", "녹십자") \
              .replace("녹십자웰빙", "녹십자") \
              .replace("GC녹십자엠에스", "녹십자") \
              .replace("GC녹십자셀", "녹십자") \
              .replace("GC녹십자랩셀", "녹십자") \
              .replace("GC녹십자MS ", "녹십자") \
              .replace("녹십자엠에스", "녹십자") \
              .replace("녹십자MS", "녹십자") \
              .replace("셀트리온GSC", "셀트리온") \
              .replace("롯데 케미칼", "롯데케미칼") \
              .replace("호남석유화학", "롯데케미칼") \
              .replace("SK 텔레콤", "SK텔레콤") \
              .replace("SKT", "SK텔레콤") \
              .replace("SK T", "SK텔레콤")

In [44]:
nnp_set = set()

In [45]:
for company_id in tqdm([1, 2, 3, 4, 6 ,7, 8, 9]):
  df_news = mysql_to_dataframe(company_id)
  content_all = ". ".join(df_news["content"])
  content_all = company_name_replace(content_all.upper())
  nnp_set = nnp_set.union(find_NNP(content_all, komoran))

  0%|          | 0/8 [00:00<?, ?it/s]C:\Users\SSAFY\AppData\Local\Temp\ipykernel_20872\3307335590.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_news = pd.read_sql(sql="SELECT * FROM news WHERE company_id={};".format(company_id), con=con, index_col="id")
 12%|█▎        | 1/8 [02:55<20:26, 175.19s/it]

 25%|██▌       | 2/8 [12:52<42:22, 423.74s/it]

 38%|███▊      | 3/8 [13:18<20:10, 242.16s/it]

 50%|█████     | 4/8 [18:55<18:38, 279.55s/it]

 62%|██████▎   | 5/8 [24:00<14:26, 288.72s/it]

 75%|███████▌  | 6/8 [25:24<07:18, 219.06s/it]

 88%|████████▊ | 7/8 [27:01<02:58, 179.00s/it]

100%|██████████| 8/8 [27:33<00:00, 206.67s/it]

In [46]:
list(nnp_set)[:5], len(nnp_set)

(['와이', '꼭짓점', '삼성전자판매', '유니티', '구관'], 12406)

In [50]:
df_result = pd.DataFrame({"nnp": list(nnp_set)})

In [51]:
df_result.head()

,nnp
0,와이
1,꼭짓점
2,삼성전자판매
3,유니티
4,구관


In [52]:
df_result.to_csv("nnp_result.csv")